Setting up a Convolutional Neural Network for image classification with Keras:

Used dataset: Traffic signs: kaggle.com/valentynsichkar/traffic-signs-preprocessed



In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import keras

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\gwehrm\AppData\Roaming\P

## Data

In [2]:
#load the data
data = pickle.load(open("data/data0.pickle","rb"))

In [3]:
# the data is a dictionary with the following keys:
# The dataset is already nicely prepared, therefore, no more preparation is needed. Even the splits
# are already done!
data.keys()

dict_keys(['x_test', 'y_validation', 'x_validation', 'labels', 'x_train', 'y_test', 'y_train'])

In [4]:
# extract the datasets
x_test = data["x_test"]
y_validation = data["y_validation"]
x_validation = data["x_validation"]
labels = data["labels"]
x_train = data["x_train"]
y_test = data["y_test"]
y_train = data["y_train"]

In [5]:
# delete the now obsolete original dataset
del(data)

In [6]:
# the labels are not yet in a one-hot encoding 
display(y_test[1])

# the labels are integers - therefore, the keras function to_categorical can be used to encode it
from keras.utils import to_categorical

# transformation:
def ohe(labs):
    ohe_labs = to_categorical(labs)
    return ohe_labs
y_test = ohe(y_test)
display(y_test[1])

11

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [7]:
# Nice, and the same with the others:
y_validation = ohe(y_validation)
y_train = ohe(y_train)

In [8]:
# the shape of the training data
x_train.shape

(86989, 3, 32, 32)

In [9]:
x_train = np.transpose(x_train/255.,[0,2,3,1])
x_test = np.transpose(x_test/255.,[0,2,3,1])
x_validation = np.transpose(x_validation/255.,[0,2,3,1])

In [10]:
# total number of classes:
len(y_train[1])

43

## Model building

In [20]:
# Import the necessary components from Keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout

# Initialize the model object
model = Sequential()

# Add a convolutional layer
model.add(Conv2D(10, kernel_size=3, activation='relu', 
               input_shape=(32,32,3),
                padding = "same"))

model.add(Dropout(0.1))


model.add(Conv2D(10, kernel_size=6, activation='relu'))

model.add(Dropout(0.1))

model.add(Conv2D(10, kernel_size=9, activation='relu'))

# model.add(Conv2D(10, kernel_size=12, activation='relu'))


# Flatten the output of the convolutional layer
model.add(Flatten())
# Add an output layer for the categories
model.add(Dense(43, activation='softmax'))

In [21]:
# Compile the model 
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Fit the model on a training set
model.fit(x_train, y_train, 
          validation_split=0.2, 
          epochs=3, batch_size=100)

Train on 69591 samples, validate on 17398 samples
Epoch 1/3
69591/69591 [==============================] - 389s 6ms/step - loss: 0.6401 - accuracy: 0.8408 - val_loss: 0.1726 - val_accuracy: 0.9619
Epoch 2/3
69591/69591 [==============================] - 371s 5ms/step - loss: 0.1217 - accuracy: 0.9692 - val_loss: 0.0780 - val_accuracy: 0.9803
Epoch 3/3
69591/69591 [==============================] - 387s 6ms/step - loss: 0.0655 - accuracy: 0.9830 - val_loss: 0.0533 - val_accuracy: 0.9866


In [22]:
# evaluate the model:
model.evaluate(x_validation,y_validation)

4410/4410 [==============================] - 6s 1ms/step


[0.3249582879329755, 0.9383220076560974]

In [23]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 10)        280       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 10)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 10)        3610      
_________________________________________________________________
dropout_7 (Dropout)          (None, 27, 27, 10)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 19, 19, 10)        8110      
_________________________________________________________________
flatten_4 (Flatten)          (None, 3610)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 43)               

In [27]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

array([[0.22745098, 0.2       , 0.16470589],
       [0.23137255, 0.2       , 0.15294118],
       [0.23921569, 0.18431373, 0.14117648],
       [0.22745098, 0.18039216, 0.14509805],
       [0.22352941, 0.20784314, 0.18039216],
       [0.1882353 , 0.1764706 , 0.16470589],
       [0.23529412, 0.19215687, 0.2       ],
       [0.19607843, 0.18039216, 0.1882353 ],
       [0.20392157, 0.1882353 , 0.1764706 ],
       [0.21960784, 0.21176471, 0.1882353 ],
       [0.21960784, 0.20784314, 0.19607843],
       [0.22745098, 0.18431373, 0.17254902],
       [0.41568628, 0.3372549 , 0.32156864],
       [0.76862746, 0.80784315, 0.83137256],
       [0.76862746, 0.81960785, 0.89411765],
       [0.7647059 , 0.8235294 , 0.9019608 ],
       [0.7411765 , 0.78039217, 0.9607843 ],
       [0.654902  , 0.6901961 , 0.67058825],
       [0.39215687, 0.4862745 , 0.5568628 ],
       [0.57254905, 0.56078434, 0.6666667 ],
       [0.7764706 , 0.79607844, 0.79607844],
       [0.77254903, 0.8156863 , 0.9529412 ],
       [0.

In [42]:
from keras import backend as K

layer_name = 'conv2d_11'
filter_index = 0  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:, :, :, filter_index])

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, x_test[0])[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# # this function returns the loss and grads given the input picture
# iterate = K.function([x_test[0][0]], [loss, grads])

ValueError: Tried to convert 'x' to a tensor and failed. Error: None values not supported.